# Início

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim
from torchsummary import summary

import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
# from tqdm import tqdm_notebook as tqdm
import time
import os
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
!wget https://zenodo.org/records/11992126/files/standardized_view.zip?download=1

--2024-10-23 15:18:03--  https://zenodo.org/records/11992126/files/standardized_view.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.103.159, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 240637095 (229M) [application/octet-stream]
Saving to: ‘standardized_view.zip?download=1’

standardized_view.z 100%[===================>] 229.49M  16.7MB/s    in 1m 45s  

2024-10-23 15:19:50 (2.18 MB/s) - ‘standardized_view.zip?download=1’ saved [240637095/240637095]



In [4]:
old_name = r"/content/standardized_view.zip?download=1"
new_name = r"/content/standardized_view.zip"
os.rename(old_name, new_name)

In [5]:
newpath = r'/content/DAGHAR'
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [6]:
!unzip /content/standardized_view.zip -d /content/DAGHAR

Archive:  /content/standardized_view.zip
   creating: /content/DAGHAR/KuHar/
  inflating: /content/DAGHAR/KuHar/validation.csv  
  inflating: /content/DAGHAR/KuHar/train.csv  
  inflating: /content/DAGHAR/KuHar/test.csv  
   creating: /content/DAGHAR/MotionSense/
  inflating: /content/DAGHAR/MotionSense/validation.csv  
  inflating: /content/DAGHAR/MotionSense/train.csv  
  inflating: /content/DAGHAR/MotionSense/test.csv  
   creating: /content/DAGHAR/RealWorld_thigh/
  inflating: /content/DAGHAR/RealWorld_thigh/validation.csv  
  inflating: /content/DAGHAR/RealWorld_thigh/train.csv  
  inflating: /content/DAGHAR/RealWorld_thigh/test.csv  
   creating: /content/DAGHAR/RealWorld_upperarm/
  inflating: /content/DAGHAR/RealWorld_upperarm/validation.csv  
  inflating: /content/DAGHAR/RealWorld_upperarm/train.csv  
  inflating: /content/DAGHAR/RealWorld_upperarm/test.csv  
   creating: /content/DAGHAR/RealWorld_waist/
  inflating: /content/DAGHAR/RealWorld_waist/validation.csv  
  inflating

In [17]:
dataNames = os.listdir(newpath)
dataNames

['UCI',
 'RealWorld_thigh',
 'MotionSense',
 'WISDM',
 'RealWorld_waist',
 'KuHar',
 'RealWorld_upperarm']

In [27]:
WISDM = pd.read_csv(newpath + '/' + dataNames[3] + '/train.csv')
WISDM.head()

,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gyro-z-57,gyro-z-58,gyro-z-59,index,activity code,user,timestamp-gyro,window,timestamp-accel,standard activity code
0,0.550232,3.188394,0.201385,-1.866123,-2.763684,1.186544,0.705521,-0.818089,-0.305727,-0.980729,...,1.023751,-0.551919,-0.847473,2913097.0,A,1644,1.819452e+15,14,1.819452e+15,2
1,-0.845642,2.849746,0.819272,1.752620,0.024921,-0.872890,-1.068011,-0.340189,-0.762406,-2.374736,...,0.085791,0.646415,1.101547,2201230.0,A,1633,6.948264e+14,15,6.948257e+14,2
2,-0.529279,-1.312229,0.267284,0.740674,0.237883,-0.537407,0.744115,0.517098,-0.952201,2.502723,...,-0.648634,-0.447404,-0.189661,1427244.0,A,1622,1.149830e+15,54,1.149830e+15,2
3,1.412971,1.946926,0.890790,-0.447067,0.210172,1.033004,0.129366,-2.512947,-3.948992,-3.960705,...,-5.113045,-4.974048,-0.868057,2524602.0,A,1638,1.140315e+15,13,1.140315e+15,2
4,-1.974706,0.836329,0.334619,-0.346837,0.531329,0.446542,-0.607665,-2.478647,1.740978,-1.619372,...,-0.022066,0.140561,0.202345,1814986.0,A,1628,1.138534e+15,47,1.138533e+15,2


In [28]:
SAC = ['Sit', 'Stand', 'Walk', 'Upstairs', 'Downstairs', 'Run']

In [29]:
aux = WISDM.values[:,:360]
y = WISDM.values[:,-1]
i = np.random.randint(0, aux.shape[0])
px.line(aux[i].reshape(6,60).T, title=str(i)+' '+SAC[int(y[i])])

In [30]:
X_train = aux.reshape(-1,6,60)
i = 1024
px.line(X_train[i].T, title=str(i)+' '+SAC[int(y[i])])

In [31]:
X_train.shape, y.shape

((8748, 6, 60), (8748,))

In [32]:
input_shape = X_train.T.shape[:-1]
input_shape

(60, 6)

In [33]:
generated_sample_shape = input_shape
noise_shape = 100
input_layer = noise_shape

n_layers = 4
kernel_size = 10
strides = 2
g_units_base=32

steps = generated_sample_shape[0]
layer_steps = [steps]
for i in range(n_layers):
    layer_steps.append(int(np.ceil(float(layer_steps[-1]) / float(strides))))
layer_steps.reverse()

conv_units = []
if n_layers > 1:
    conv_units.append(g_units_base)
    for _ in range(n_layers - 2):  # minus the first and the last layers
        conv_units.append(conv_units[-1] * 2)
conv_units.reverse()
# the last layer must be aligned to the number of dimensions of input.
conv_units.append(generated_sample_shape[-1])

In [34]:
conv_units, layer_steps

([128, 64, 32, 6], [4, 8, 15, 30, 60])

In [35]:
class Genera(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(input_layer, layer_steps[0] * conv_units[0] * 2, device=device)
        self.l2 = nn.BatchNorm1d(layer_steps[0] * conv_units[0] * 2, device=device)
        self.l3 = nn.ConvTranspose1d(conv_units[0] * 2, conv_units[0], kernel_size,
                                    stride=strides, padding=4, device=device)
        self.l4 = nn.BatchNorm1d(conv_units[0], device=device)
        self.l5 = nn.ConvTranspose1d(conv_units[0], conv_units[1], kernel_size,
                                    stride=strides, padding=4, device=device)
        self.l6 = nn.BatchNorm1d(conv_units[1], device=device)
        self.l7 = nn.ConvTranspose1d(conv_units[1], conv_units[2], kernel_size,
                                    stride=strides, padding=4, device=device)
        self.l8 = nn.BatchNorm1d(conv_units[2], device=device)
        self.l9 = nn.ConvTranspose1d(conv_units[2], conv_units[3], kernel_size,
                                    stride=strides, padding=6, output_padding=0, device=device)

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = F.relu(x)
        x = x.reshape(x.shape[0], layer_steps[0], conv_units[0] * 2)
        x = torch.transpose(x, 1, 2)
        x = self.l3(x)
        x = self.l4(x)
        x = F.relu(x)
        x = self.l5(x)
        x = self.l6(x)
        x = F.relu(x)
        x = self.l7(x)
        x = self.l8(x)
        x = F.relu(x)
        x = self.l9(x)
        return x
generator = Genera().to(device)

In [36]:
summary(generator, (100,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1024]         103,424
       BatchNorm1d-2                 [-1, 1024]           2,048
   ConvTranspose1d-3               [-1, 128, 8]         327,808
       BatchNorm1d-4               [-1, 128, 8]             256
   ConvTranspose1d-5               [-1, 64, 16]          81,984
       BatchNorm1d-6               [-1, 64, 16]             128
   ConvTranspose1d-7               [-1, 32, 32]          20,512
       BatchNorm1d-8               [-1, 32, 32]              64
   ConvTranspose1d-9                [-1, 6, 60]           1,926
Total params: 538,150
Trainable params: 538,150
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.07
Params size (MB): 2.05
Estimated Total Size (MB): 2.12
-------------------------------------------

In [37]:
units = [32]
for _ in range(n_layers - 1):  # exclude the first layer.
    units.append(units[-1] * 2)

In [38]:
units

[32, 64, 128, 256]

In [39]:
class Discri(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Conv1d(input_shape[-1], units[0], kernel_size,stride=strides,padding=6, device=device)
        self.l2 = nn.Conv1d(units[0], units[1], kernel_size,stride=strides,padding=4, device=device)
        self.l3 = nn.Conv1d(units[1], units[2], kernel_size,stride=strides,padding=4, device=device)
        self.l4 = nn.BatchNorm1d(units[2], device=device)
        self.l5 = nn.Conv1d(units[2], units[3], kernel_size,stride=strides,padding=4, device=device)
        self.l6 = nn.BatchNorm1d(units[3], device=device)
        self.l7 = nn.Linear(layer_steps[0] * units[-1], 1, device=device)

    def forward(self, x):
        x = self.l1(x)
        x = F.leaky_relu(x, 0.2)
        x = self.l2(x)
        x = F.leaky_relu(x, 0.2)
        x = self.l3(x)
        x = self.l4(x)
        x = F.leaky_relu(x, 0.2)
        x = self.l5(x)
        x = self.l6(x)
        x = F.leaky_relu(x, 0.2)
        x = torch.transpose(x, 1, 2)
        x = torch.flatten(x, start_dim=1)
        x = self.l7(x)
        return x
discriminator = Discri().to(device)

In [40]:
summary(discriminator, (6,60))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 32, 32]           1,952
            Conv1d-2               [-1, 64, 16]          20,544
            Conv1d-3               [-1, 128, 8]          82,048
       BatchNorm1d-4               [-1, 128, 8]             256
            Conv1d-5               [-1, 256, 4]         327,936
       BatchNorm1d-6               [-1, 256, 4]             512
            Linear-7                    [-1, 1]           1,025
Total params: 434,273
Trainable params: 434,273
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 1.66
Estimated Total Size (MB): 1.70
----------------------------------------------------------------


In [41]:
generator.l1.weight = nn.init.trunc_normal_(generator.l1.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l1.bias.data.zero_()
generator.l2.weight = nn.init.trunc_normal_(generator.l2.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l2.bias.data.zero_()
generator.l3.weight = nn.init.trunc_normal_(generator.l3.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l3.bias.data.zero_()
generator.l4.weight = nn.init.trunc_normal_(generator.l4.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l4.bias.data.zero_()
generator.l5.weight = nn.init.trunc_normal_(generator.l5.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l5.bias.data.zero_()
generator.l6.weight = nn.init.trunc_normal_(generator.l6.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l6.bias.data.zero_()
generator.l7.weight = nn.init.trunc_normal_(generator.l7.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l7.bias.data.zero_()
generator.l8.weight = nn.init.trunc_normal_(generator.l8.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l8.bias.data.zero_()
generator.l9.weight = nn.init.trunc_normal_(generator.l9.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
generator.l9.bias.data.zero_()

tensor([0., 0., 0., 0., 0., 0.], device='cuda:0')

In [42]:
discriminator.l1.weight = nn.init.trunc_normal_(discriminator.l1.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l1.bias.data.zero_()
discriminator.l2.weight = nn.init.trunc_normal_(discriminator.l2.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l2.bias.data.zero_()
discriminator.l3.weight = nn.init.trunc_normal_(discriminator.l3.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l3.bias.data.zero_()
discriminator.l4.weight = nn.init.trunc_normal_(discriminator.l4.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l4.bias.data.zero_()
discriminator.l5.weight = nn.init.trunc_normal_(discriminator.l5.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l5.bias.data.zero_()
discriminator.l6.weight = nn.init.trunc_normal_(discriminator.l6.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l6.bias.data.zero_()
discriminator.l7.weight = nn.init.trunc_normal_(discriminator.l7.weight, mean=0.0, std=0.02, a=-0.04, b=0.04)
discriminator.l7.bias.data.zero_()

tensor([0.], device='cuda:0')

In [43]:
X_train = torch.tensor(X_train.astype(np.float32), dtype=torch.float32, device=device).detach()

In [44]:
generator = generator.to(device)
discriminator = discriminator.to(device)

# torch.manual_seed(42)

loss_fc = nn.BCEWithLogitsLoss()
sig = nn.Sigmoid()

g_opt = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_opt = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

n_epochs = 300   # number of epochs to run
batch_size = 128  # size of each batch
batch_start = torch.arange(0, len(X_train), batch_size)

dLoss = []
gLoss = []
acc = []

z_batch = torch.randn(X_train.shape[0], noise_shape, device=device).detach()
generated_samples = generator(z_batch)
fake_logits = discriminator(generated_samples)
real_logits = discriminator(X_train)
real_loss = loss_fc(real_logits, torch.ones_like(real_logits))
fake_loss = loss_fc(fake_logits, torch.zeros_like(fake_logits))
d_loss = real_loss + fake_loss
g_loss = loss_fc(fake_logits, torch.ones_like(fake_logits))
gLoss.append(float(g_loss))
dLoss.append(float(d_loss))

real_prob = sig(real_logits)
fake_prob = sig(fake_logits)
y_real = torch.ones_like(real_prob)
y_fake = torch.zeros_like(fake_prob)
y = torch.cat((y_real, y_fake), axis=0)
y_pred = torch.cat((real_prob, fake_prob), axis=0)
acc.append(float(torch.mean(((y_pred>.5).double() == y).double())))

# training loop
for epoch in range(n_epochs):
    generator.train()
    discriminator.train()
    with tqdm(batch_start, unit="batch", mininterval=0, disable=False) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train[start:start+batch_size,:,:]
            z_batch = torch.randn(batch_size, noise_shape, device=device).detach()
            # generator pass
            generated_samples = generator(z_batch)
            fake_logits = discriminator(generated_samples)
            g_loss = loss_fc(fake_logits, torch.ones_like(fake_logits))
            g_opt.zero_grad()
            g_loss.backward()
            g_opt.step()
            # discriminator pass
            generated_samples = generator(z_batch)
            fake_logits = discriminator(generated_samples)
            real_logits = discriminator(X_batch)
            # check acc
            real_prob = sig(real_logits)
            fake_prob = sig(fake_logits)
            y_real = torch.ones_like(real_prob)
            y_fake = torch.zeros_like(fake_prob)
            y = torch.cat((y_real, y_fake), axis=0)
            y_pred = torch.cat((real_prob, fake_prob), axis=0)
            if torch.mean(((y_pred>.5).double() == y).double()) <= 0.75:
                real_loss = loss_fc(real_logits, torch.ones_like(real_logits))
                fake_loss = loss_fc(fake_logits, torch.zeros_like(fake_logits))
                d_loss = real_loss + fake_loss
                d_opt.zero_grad()
                d_loss.backward()
                d_opt.step()
            # print progress
            bar.set_postfix(loss=float(g_loss))
    generator.eval()
    discriminator.eval()
    z_batch = torch.randn(X_train.shape[0], noise_shape, device=device).detach()
    generated_samples = generator(z_batch)
    fake_logits = discriminator(generated_samples)
    real_logits = discriminator(X_train)
    real_loss = loss_fc(real_logits, torch.ones_like(real_logits))
    fake_loss = loss_fc(fake_logits, torch.zeros_like(fake_logits))
    d_loss = real_loss + fake_loss
    g_loss = loss_fc(fake_logits, torch.ones_like(fake_logits))
    gLoss.append(float(g_loss))
    dLoss.append(float(d_loss))
    real_prob = sig(real_logits)
    fake_prob = sig(fake_logits)
    y_real = torch.ones_like(real_prob)
    y_fake = torch.zeros_like(fake_prob)
    y = torch.cat((y_real, y_fake), axis=0)
    y_pred = torch.cat((real_prob, fake_prob), axis=0)
    acc.append(float(torch.mean(((y_pred>.5).double() == y).double())))

Epoch 299: 100%|██████████| 69/69 [00:01<00:00, 54.33batch/s, loss=1.02]


In [45]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=dLoss, mode="lines", showlegend=True, name='dLoss'))
fig.add_trace(go.Scatter(y=gLoss, mode="lines", showlegend=True, name='gLoss'))
fig.add_trace(go.Scatter(y=acc, mode="lines", showlegend=True, name='acc'))
fig.update_layout(width=1000, height=600)

fig.show()

In [46]:
z_batch = torch.randn(batch_size, noise_shape, device=device).detach()
generated_samples = generator(z_batch).cpu().detach().numpy()

In [48]:
i += 1
px.line(generated_samples[i,:,:].T)

In [ ]:
class Encod(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Conv1d(input_shape[-1], units[0], kernel_size,stride=strides,padding=6, device=device)
        self.l2 = nn.Conv1d(units[0], units[1], kernel_size,stride=strides,padding=4, device=device)
        self.l3 = nn.Conv1d(units[1], units[2], kernel_size,stride=strides,padding=4, device=device)
        self.l4 = nn.BatchNorm1d(units[2], device=device)
        self.l5 = nn.Conv1d(units[2], units[3], kernel_size,stride=strides,padding=4, device=device)
        # self.l6 = nn.BatchNorm1d(units[3], device=device)
        # self.l7 = nn.Linear(layer_steps[0] * units[-1], 1, device=device)

    def forward(self, x):
        x = self.l1(x)
        x = F.leaky_relu(x, 0.2)
        x = self.l2(x)
        x = F.leaky_relu(x, 0.2)
        x = self.l3(x)
        x = self.l4(x)
        x = F.leaky_relu(x, 0.2)
        x = self.l5(x)
        # x = self.l6(x)
        # x = F.leaky_relu(x, 0.2)
        # x = torch.transpose(x, 1, 2)
        # x = torch.flatten(x, start_dim=1)
        # x = self.l7(x)
        return x
encoder = Encod().to(device)

# Próximos passos